In [4]:
import math, torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torch.distributions as distributions
import collections, time
import h5py, abc, os, pdb
import matplotlib.pyplot as plt
from scipy import linalg, signal
torch.set_default_tensor_type(torch.DoubleTensor)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers,Input
from keras.models import  Sequential,load_model
from keras.optimizers import SGD, Adam
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.regularizers import L1L2, L2
import keras.backend as K
from sklearn.model_selection import train_test_split
from package.Graphs import Graphs

In [5]:
state0=np.load('../Data/neel_readout/Q1_2us/state0.npy')[:,700:3700]
state1=np.load('../Data/neel_readout/Q1_2us/state1.npy')[:,700:3700]
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)

(6000, 3000)


In [7]:
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

min,max= -(2**16),(2**16)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))
x= (x-min)/(max-min)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())
print(x_train.shape)

x_train=x_train.reshape((x_train.shape[0],int(x_train.shape[1]/10),10))
x_test=x_test.reshape((x_test.shape[0],int(x_test.shape[1]/10),10))
x_train=np.mean(x_train,axis=2)
x_test=np.mean(x_test,axis=2)
x_train=x_train[:,:,None]
x_test=x_test[:,:,None]
print(x_train.shape,x_test.shape)

(12000, 3000)
(12000,)
Min: -684.000, Max: 688.000
Min: 0.495, Max: 0.505
(8400, 3000)
4233.0
(8400, 3000)
(8400, 300, 1) (3600, 300, 1)


In [16]:
print(x_train.shape,y_train.shape)
y_train=y_train[:,None]
y_test=y_test[:,None]
train_ds = data.TensorDataset(torch.tensor(x_train),torch.tensor( y_train))
test_ds = data.TensorDataset(torch.tensor(x_test),torch.tensor( y_test))

(8400, 300, 1) (8400, 1, 1)


In [28]:
class GRU(nn.Module):
    def __init__(self, hidden_size=64, num_layers=2, batch_first=True, dropout=0, bidirectional=True): 
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.num_directions = 2 if bidirectional else 1
        self.dropout = dropout
        self.gru = nn.GRU(1, hidden_size, num_layers=num_layers, 
                            batch_first=batch_first, dropout=dropout, bidirectional=bidirectional)
        self.linear = nn.Linear(self.num_directions * hidden_size, 1)
        self.softmax = nn.Softmax(dim=2)
        self.loss_func = nn.CrossEntropyLoss()

        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            raise IOError('No GPU')

        self.to(self.device)
#         self.reset()

    def train_gru(self,train_ds, val_ds, optimizer, epochs=30, train_batch_size=128, val_batch_size=2048, seq_size=100000,
                load_dict=None, save_path=None, loaded_save_dict=None, tune=False):
        train_dl = data.DataLoader(train_ds, batch_size=train_batch_size, shuffle=True)
        val_dl = data.DataLoader(test_ds, batch_size=val_batch_size)
        save_dict = {}
        if loaded_save_dict:
            start_epoch = loaded_save_dict['epoch'] + 1
            if start_epoch >= epochs:
                print('No training. Start epoch larger than epochs')
                return None
            optimizer.load_state_dict(loaded_save_dict['optimizer_state_dict'])
            log = loaded_save_dict['log']
            save_dict = loaded_save_dict
        else:
            start_epoch = 0
            log = {}
            log['train_loss'], log['train_acc'] = torch.zeros(0), torch.zeros(0)
            log['val_loss'], log['val_acc'], log['final_fid'] =  torch.zeros(0), torch.zeros(0), torch.zeros(0)

        train_loss, train_acc, val_loss, val_acc, final_fid = 0.0, 0.0, 0.0, 0.0, 0.0

        for epoch in tqdm(range(start_epoch, epochs), leave=False):
            self.train()
            train_steps = 0
            for inputs, states in train_dl:
                num_steps = inputs.shape[1]
                h_out = self.init_h(batch_size=len(inputs))
                inputs, states = inputs.to(self.device), states.to(self.device)

                optimizer.zero_grad()
                logits, h_out = self(inputs_curr, h0)
                loss = self.loss_func(logits.view(-1, ERROR_SIZE), states_curr.reshape(-1))
                loss.backward(retain_graph=True)
                optimizer.step()

                probabilities = self.softmax(logits)
                train_acc += (torch.argmax(probabilities.view(-1, ERROR_SIZE), dim=1) == states_curr.reshape(-1)).sum().item()
                train_loss += loss.item()    
                train_steps += inputs_curr.size(0) * inputs_curr.size(1)

                train_loss = train_loss / len(train_dl)
                train_acc = train_acc / train_steps



In [31]:
gru=GRU()
train_gru(train_ds, test_ds, optimizer=optim.Adam(gru.parameters(), epochs=30, train_batch_size=64, val_batch_size=64)

SyntaxError: unexpected EOF while parsing (1510643265.py, line 2)